In [1]:
import numpy as np
import torch
from pinnies import *
from torch import nn

In [2]:
class Problem(Fredholm1D):
    def __init__(self, domain, num_train, model):
        super().__init__(domain, num_train)
        self.a, self.b = domain
        self.model = model
        self.F = torch.exp(self.x) + self.x - 4 / 3
        self.K = self.T

    def residual(self):
        y = self.predict(self.x)
        zeta = y
        I = self.quad(self.K.T * zeta, self.a, self.b)
        return [y - I - self.F]

    def exact(self, x):
        # For validation
        return x + torch.exp(x)


set_seed(42)

model = nn.Sequential(
    nn.Linear(1, 10),
    nn.Tanh(),
    nn.Linear(10, 1),
)

p = Problem([0, 1], 30, model)
p.solve(10)

Validation: 3.28e-04: 100%|█████████████████████████████████████████████████████████████████████████████████████████████| 10/10 [00:00<00:00, 41.62it/s]


tensor(0.0003)

In [5]:
class Problem(Volterra1D):
    def __init__(self, domain, num_train, model):
        super().__init__(domain, num_train)
        self.a, self.b = domain
        self.model = model
        self.K = torch.exp(self.T - self.X)
        self.zero = torch.tensor([[0.0]])

    def residual(self):
        y = self.predict(self.x)
        y_x = self.diff(y, self.x, n=1)

        u_t = self.predict_on_T()
        zeta = u_t
        I = self.quad(self.K * zeta, self.a, self.x)

        initial = self.get_initial()
        return [y_x + y - I, initial]

    def get_initial(self):
        return self.predict(self.zero) - (1)

    def exact(self, x):
        # For validation
        return torch.exp(-x) * torch.cosh(x)


set_seed(42)

model = nn.Sequential(
    nn.Linear(1, 10),
    nn.Tanh(),
    nn.Linear(10, 10),
    nn.Tanh(),
    nn.Linear(10, 1),
)

p2 = Problem((0, 5), 10, model)
p2.solve(10, learning_rate=0.3)

Validation: 1.82e-04: 100%|█████████████████████████████████████████████████████████████████████████████████████████████| 10/10 [00:00<00:00, 25.82it/s]


tensor(0.0002)

In [8]:
class Problem(Fredholm2D):
    def __init__(self, domain, num_train, model):
        super().__init__(domain, num_train)
        self.a, self.b = domain[0]
        self.c, self.d = domain[1]
        self.model = mlp
        source = lambda x, y: (x**2 * y) + ((4 / 9) * x)
        self.F = source(self.x[:, 0], self.x[:, 1]).reshape(self.N, self.N)
        self.K = -(1 / 2 * self.X * self.T)

    def residual(self):
        y = self.predict(self.x).reshape(self.N, self.N)
        zeta = y

        I = self.quad(self.K * zeta, self.c, self.d)
        I = self.quad(I, self.a, self.b)
        return [y - I - self.F]

    def exact(self, X):
        # For validation
        x, y = X[:, 0], X[:, 1]
        return x**2 * y


set_seed(42)

mlp = nn.Sequential(
    nn.Linear(2, 10),
    nn.Tanh(),
    nn.Linear(10, 10),
    nn.Tanh(),
    nn.Linear(10, 1),
)

p1 = Problem([(0, 1), (0, 2)], 25, mlp)
p1.solve(100)

Validation: 4.35e-04: 100%|███████████████████████████████████████████████████████████████████████████████████████████| 100/100 [00:09<00:00, 11.03it/s]


tensor(0.0003)

In [12]:
class Problem(Volterra2D):
    def __init__(self, domain, num_train, model):
        super().__init__(domain, num_train)
        self.a, self.b = domain[0]
        self.c, self.d = domain[1]
        self.model = mlp
        source = lambda x, y: (
            x
            + y
            + torch.exp(x + y)
            * (
                x * torch.exp(x + y)
                + y * torch.exp(x + y)
                - 2 * torch.exp(x + y)
                - torch.exp(x) * x
                - torch.exp(y) * y
                + 2 * torch.exp(x)
                + 2 * torch.exp(y)
                - 2
            )
        )
        self.F = source(self.x[:, 0], self.x[:, 1]).reshape(self.N, self.N)
        self.K = torch.exp(self.S + self.T) * torch.exp(self.X + self.Y)

    def residual(self):
        y = self.predict(self.x).reshape(self.N, self.N)
        zeta = self.predict_on_ST()

        I = self.quad(self.K * zeta, self.c, self.bdy)

        I = self.quad(I.permute(2, 1, 0), self.a, self.bdx.reshape(1, -1))

        return [y + I - self.F]

    def exact(self, X):
        # For validation
        x, y = X[:, 0], X[:, 1]
        return x + y


set_seed(42)

mlp = nn.Sequential(
    nn.Linear(2, 10),
    nn.Tanh(),
    nn.Linear(10, 10),
    nn.Tanh(),
    nn.Linear(10, 1),
)

p1 = Problem([(0, 1), (0, 2)], 15, mlp)
p1.solve(20)

Validation: 1.20e-02: 100%|█████████████████████████████████████████████████████████████████████████████████████████████| 20/20 [00:04<00:00,  4.16it/s]


tensor(0.0052)

In [15]:
class Problem(Fredholm3D):
    def __init__(self, domain, num_train, model):
        super().__init__(domain, num_train)
        self.a, self.b = domain[0]
        self.c, self.d = domain[1]
        self.e, self.f = domain[2]

        self.model = mlp
        source = lambda x, y, z: (
            x**2 * y * torch.exp(z) - ((-torch.e + np.exp(2)) * np.exp(-1)) / 2
        )
        self.F = source(self.x[:, 0], self.x[:, 1], self.x[:, 2]).reshape(
            self.N, self.N, self.N
        )
        self.K = torch.exp(self.S) * self.R

    def residual(self):
        y = self.predict(self.x).reshape(self.N, self.N, self.N)
        zeta = y
        I = self.quad(self.K * zeta, self.e, self.f)
        I = self.quad(I, self.c, self.d)
        I = self.quad(I, self.a, self.b)
        return [y - I - self.F]

    def exact(self, X):
        # For validation
        x, y, z = X[:, 0], X[:, 1], X[:, 2]
        return x**2 * y * torch.exp(z)


set_seed(42)

mlp = nn.Sequential(
    nn.Linear(3, 10),
    nn.Tanh(),
    nn.Linear(10, 10),
    nn.Tanh(),
    nn.Linear(10, 1),
)

p1 = Problem([(0, 1), (-1, 1), (1, 2)], 10, mlp)
p1.solve(10)

Validation: 2.48e-02: 100%|█████████████████████████████████████████████████████████████████████████████████████████████| 10/10 [00:02<00:00,  4.69it/s]


tensor(0.0474)

In [18]:
class Problem(Volterra1D, Fractional):
    def __init__(self, domain, num_train, model):
        super().__init__(domain, num_train)
        self.a, self.b = domain
        self.model = model
        self.kappa = 2
        self.u0 = 0.1
        self.K = torch.ones_like(self.T)
        self.M = self.fracmatrix(0.5)

    def residual(self):
        y = self.predict(self.x)
        y_x = self.diff(y, self.x, n=1)

        u_t = self.predict_on_T()
        zeta = u_t
        I = self.quad(self.K * zeta, self.a, self.x)

        initial = self.get_initial()
        return [self.kappa * (self.M @ y) - y + y**2 + y * I, 1e1 * initial]

    def get_initial(self):
        zero = torch.tensor([[0.0]])
        return self.predict(zero) - self.u0

    def criteria(self):
        x_test = torch.linspace(self.a, self.b, 10000).reshape(-1, 1)
        predict = (self.predict(x_test)).detach()
        x_test = x_test.detach().numpy()
        predict = predict.detach().numpy()

        u_max_pred = np.max(predict)
        u_max_exact = 1 + self.kappa * np.log(self.kappa / (1 + self.kappa - self.u0))

        return np.abs(u_max_exact - u_max_pred)


set_seed(42)

model = nn.Sequential(
    nn.Linear(1, 10),
    nn.Tanh(),
    nn.Linear(10, 10),
    nn.Tanh(),
    nn.Linear(10, 1),
)

p2 = Problem((0, 5), 20, model)

p2.solve(10, learning_rate=0.1)

print(p2.criteria())

Loss: 4.17e+02: 100%|███████████████████████████████████████████████████████████████████████████████████████████████████| 10/10 [00:03<00:00,  3.13it/s]

0.07243856263208914
